# Stylegan3 from NVIDIA

* 이 파일은 stylegan3를 colab gpu서버를 활용하여 학습을 해볼 수 있는 코드를 정리한 파일입니다.
* 간단하게 실제로 실행해볼 수 있도록 설계했으므로 좀 더 자세하게 알고 싶다면 [이 곳](https://github.com/NVlabs/stylegan3)을 클릭하여 NVIDIA의 stylegan3을 참고하면 됩니다.
* 학습에 사용할 이미지를 필요로 하므로 **반드시** 구하여 사용하시기 바랍니다.

학습은 아래와 같이 git clone, 데이터 전처리, 학습 총 3단계로 진행됩니다.

## 1. git clone
* 학습에 앞서 필요한 패키지를 설치할 필요가 있습니다. 아래 코드를 실행합니다.


In [ ]:
!pip install einops ninja gdown torch==1.10.0

* colab은 구글 드라이브와 연동하여 사용할 수 있습니다. 
* 아래 코드를 순차적으로 실행한다면 구글 드라이브에 stylegan을 저장하고 활용할 수 있습니다.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/bvshyam/stylegan3.git /content/drive/MyDrive/stylegan3/

# 2. 데이터 전처리
* 학습에 사용할 데이터를 전처리 할 필요가 있습니다. 먼저 이미지가 들어있는  폴더를 내드라이브에 올리고 파일이 위치한 경로를 가져옵니다.
* 그 후 아래 코드를 실행하면 됩니다.
*아래 코드는 stylegan 폴더 내 data폴더를 만들고 그 안에 저장한 cats폴더속 cat이미지를 활용하여 전처리를 하는 코드입니다.

* 변수 설명

data_dir : 훈련에 사용할 이미지들이 위차한 폴더 경로

dest : 전처리한 데이터를 저장할 폴더 경로와 파일이름

resolution : 전처리 후 저장될 이미지 크기 ex) 256x256 입력시 가로 256 세로 256 크기의 이미지로 전처리 됨.

In [ ]:
data_dir = '/content/drive/MyDrive/stylegan3/data/cats/'
dest = '/content/drive/MyDrive/stylegan3/data/cats-256x256.zip'
resolution = '256x256'

!python /content/drive/MyDrive/stylegan3/dataset_tool.py --source=$data_dir --dest=$dest --resolution=$resolution


# 학습
* 마지막 학습 단계입니다. 전처리한 데이터를 이용하여 학습을 할 수 있습니다.
* 파일 경로를 잘 설정하면 문제없이 학습 가능합니다.

* 변수 설명

out_dir : 결과 데이터를 저장할 경로

data : 학습에 사용할 전처리된 데이터의 경로

gpus : 학습에 사용할 gpu 갯수, 최대8까지 가능

cfg : stylegan3-t, stylegan3-r, stylegan2 선택 가능

batch : 전체 batch 크기

gamma : R1 정규화 가중치

batch-gpu : gpu당 batch 크기, 1개의 gpu만 쓴다면 따로 설정할 필요는 없습니다.

kimg : 전체 학습 기간을 결정하는 값.

tick : 몇 kimg마다 진행상황을 보여줄것인지 결정하는 값

snap : 몇 tick마다 학습 과정을 저장할것인지 결정하는 값

resume : 학습에 사용할 pickle 파일이 위치한 경로

freezed : 고정할 D의 시작 층

In [ ]:
out_dir = '/content/drive/MyDrive/stylegan3/results/'
data = '/content/drive/MyDrive/stylegan3/data/cats-256x256.zip'
gpus = 1
cfg = 'stylegan3-t'
batch = 8
gamma = 50
batch-gpu = 8
kimg = 100
tick = 1
snap = 10
resume = 'https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-ffhqu-256x256.pkl'
freezed = 4

!python train.py --outdir=$out_dir --cfg=$cfg --data=$data \
  --gpus=$gpus --batch=$batch --batch-gpu=$batch-gpu --gamma=$gamma --kimg=$kimg --snap=$snap --tick=$tick \
  --resume=$resume --freezed=$freezed

# 유사도 계산
* 생성된 이미지가 얼마나 실제와 유사한지 판단하고 싶다면 아래 코드에 변수값을 잘 설정하고 실행하면 됩니다.

* 변수 설명


metrics : 사용할 유사도 측정법

data : 유사도 측정에 사용할 전처리된 데이터의 경로

network : 유사도 측정에 사용할 모델이 있는 pickle파일 경로


In [ ]:
metrics = 'fid50k_full'
data = '/content/drive/MyDrive/stylegan3/data/cats-256x256.zip'
network = 'https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-ffhqu-256x256.pkl'

!python /content/drive/MyDrive/WIP/stylegan3/calc_metrics.py --metrics=$metrics --data=$data --network=$network